In [39]:
import pandas as pd
import re
import requests 
import pyodbc
from bs4 import BeautifulSoup
from string import ascii_uppercase
import re

In [40]:
productionDB = True
year = 2022


connection = ''
conn1 = ''
conn2 = ''

if productionDB:
    connection = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=69.167.149.140,782;database=MLR990_TheWAC;uid=MLR990_wacuser;pwd=WACP@ssword123!@#;')
    conn1 = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=69.167.149.140,782;database=MLR990_TheWAC;uid=MLR990_wacuser;pwd=WACP@ssword123!@#;')
    conn2 = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=69.167.149.140,782;database=MLR990_TheWAC;uid=MLR990_wacuser;pwd=WACP@ssword123!@#;')
else:
    connection = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=(localdb)\\mssqllocaldb;database=TheWAC;')
    conn1 = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=(localdb)\\mssqllocaldb;database=TheWAC;')
    conn2 = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};server=(localdb)\\mssqllocaldb;database=TheWAC;')


In [41]:
def GetTeamId(team):
    return {
        'ARI':1,
        'CRD':1,
        'ATL':2,
        'BAL':3,
        'RAV':3,
        'BUF':4,
        'CAR':5,
        'CHI':6,
        'CIN':7,
        'CLE':8,
        'DAL':9,
        'DEN':10,
        'DET':11,
        'GNB':12,
        'HOU':13,
        'HTX':13,
        'IND':14,
        'CLT':14,
        'JAX':15,
        'KAN':16,
        'LVR':17,
        'RAI':17,
        'LAC':18,
        'SDG':18,
        'LAR':19,
        'RAM':19,
        'MIA':20,
        'MIN':21,
        'NWE':22,
        'NOR':23,
        'NYG':24,
        'NYJ':25,
        'PHI':26,
        'PIT':27,
        'SFO':28,
        'SEA':29,
        'TAM':30,
        'TEN':31,
        'OTI':31,
        'WAS':32,
        '':'',
        '2TM':''
    }[team]

In [42]:
def GetTeamCode(teamId):
    return {
        1:'ARI',
        1:'CRD',
        2:'ATL',
        3:'BAL',
        3:'RAV',
        4:'BUF',
        5:'CAR',
        6:'CHI',
        7:'CIN',
        8:'CLE',
        9:'DAL',
        10:'DEN',
        11:'DET',
        12:'GNB',
        13:'HOU',
        13:'HTX',
        14:'IND',
        14:'CLT',
        15:'JAX',
        16:'KAN',
        17:'LVR',
        17:'RAI',
        18:'LAC',
        18:'SDG',
        19:'LAR',
        19:'RAM',
        20:'MIA',
        21:'MIN',
        22:'NWE',
        23:'NOR',
        24:'NYG',
        25:'NYJ',
        26:'PHI',
        27:'PIT',
        28:'SFO',
        29:'SEA',
        30:'TAM',
        31:'TEN',
        31:'OTI',
        32:'WAS'
    }[teamId]

In [43]:
positionList = ['QB', 'RB', 'WR', 'TE', 'K']

In [25]:
cursor = connection.cursor()


for c in ascii_uppercase:
    url = 'https://www.pro-football-reference.com/players/{}/'.format(c)

    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    players = soup.find("div", {"id": "div_players"}).find_all('p')
    for player in players:
        lastActiveYear = player.get_text().split('-')[-1]
        if (int(lastActiveYear) < 2014):
            continue
        retired = True
        boldPlayer = player.find_all('b')
        if len(boldPlayer) > 0:
            retired = False;
        nameLink = player.find("a", href=True)
        playerId = nameLink['href'].split('/')[3].split('.')[0]
        fullName = nameLink.get_text()

        names = nameLink.get_text().split(" ")
        first = names[0]
        last = ''
        for name in names[1:]:
            last = last + ' ' + name
        position = player.get_text().split('(')[1][0:-1].split(')')[0]
        if position in positionList:
            playerUrl = 'https://www.pro-football-reference.com{}'.format(nameLink['href'])
            playerPage = requests.get(playerUrl)
            playerSoup = BeautifulSoup(playerPage.text, 'html.parser')
            print(playerUrl)
            jerseyContainer = playerSoup.find("div", {"class": "uni_holder"}).find_all('text');
            jerseyNumber = ''
            if len(jerseyContainer) > 0:
                jerseyNumber = jerseyContainer[-1].get_text()
            currentTeam = playerSoup.find("span", {"itemprop": "affiliation"})
            team = ''
            if currentTeam is not None:
                t = currentTeam.find("a", href=True)['href'].split('/')[2]
                team = GetTeamId(t.upper())
            birthDate = ''
            dob = playerSoup.find("span", {"itemprop": "birthDate"})
            if dob is not None:
                birthDate = dob['data-birth']
            height = ''
            h = playerSoup.find("span", {"itemprop": "height"})
            if h is not None:
                height = h.get_text()
            weight = ''
            w = playerSoup.find("span", {"itemprop": "weight"})
            if w is not None:
                weight = w.get_text()
            collegeSection = playerSoup.find_all("p")[5]
            college = collegeSection.find('strong',text = re.compile('College'))
            playerCollege = ''
            if college is not None:
                playerCollege = collegeSection.find_all('a', href=True)[0].get_text()
                
            print("{} {} {} {} {} {} {} {} {} {} {} {}".format(playerId,nameLink['href'],first.strip(),last.strip(),team,birthDate,jerseyNumber,False,position,playerCollege,jerseyNumber, birthDate,height,weight ))
            
            if not team:
                cursor.execute("INSERT INTO Players ([ProReferenceKey],[ProReferenceURL], [FullName], [FirstName], [LastName], [JerseyNumber], [Rookie], [Position], [College], [DateOfBirth], [Height], [Weight], [TradingBlock], [Retired],[WeeksLeftOnIR]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", playerId,nameLink['href'],fullName,first.strip(),last.strip(),jerseyNumber,False,position,playerCollege,birthDate,height,weight,False, retired,0 )
                cursor.commit()
            else:
                cursor.execute("INSERT INTO Players ([ProReferenceKey],[ProReferenceURL], [FullName],[FirstName], [LastName], [NFLTeamId], [JerseyNumber], [Rookie], [Position], [College], [DateOfBirth], [Height], [Weight], [TradingBlock], [Retired],[WeeksLeftOnIR]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", playerId,nameLink['href'],fullName,first.strip(),last.strip(),team,jerseyNumber,False,position,playerCollege,birthDate,height,weight,False,retired,0 )
                cursor.commit()

missingPlayers = [
    [['Thaddeus', 'Moss'],'/players/M/MossTh01.htm','TE'],prof
    [['Jabrill', 'Peppers'],'/players/P/PeppJa00.htm','SS'],
    [['Brandon', 'Wilson'],'/players/W/WilsBr01.htm','CB'],
    [['Jamal', 'Agnew'],'/players/A/AgneJa00.htm','WR'],
]

for missingPlayerInfo in missingPlayers:
    names = missingPlayerInfo[0]
    first = names[0]
    last = ''
    for name in names[1:]:
        last = last + ' ' + name
        
    missingPlayerLink = missingPlayerInfo[1]
    playerId = missingPlayerLink.split('/')[3].split('.')[0]
    playerUrl = 'https://www.pro-football-reference.com{}'.format(missingPlayerLink)
    playerPage = requests.get(playerUrl)
    playerSoup = BeautifulSoup(playerPage.text, 'html.parser')
    jerseyContainer = playerSoup.find("div", {"class": "uni_holder"}).find_all('text');
    jerseyNumber = ''
    if len(jerseyContainer) > 0:
        jerseyNumber = jerseyContainer[-1].get_text()
    currentTeam = playerSoup.find("span", {"itemprop": "affiliation"})
    team = ''
    if currentTeam is not None:
        t = currentTeam.find("a", href=True)['href'].split('/')[2]
        team = GetTeamId(t.upper())
    birthDate = ''
    dob = playerSoup.find("span", {"itemprop": "birthDate"})
    if dob is not None:
        birthDate = dob['data-birth']
    height = ''
    h = playerSoup.find("span", {"itemprop": "height"})
    if h is not None:
        height = h.get_text()
    weight = ''
    w = playerSoup.find("span", {"itemprop": "weight"})
    if w is not None:
        weight = w.get_text()
    collegeSection = playerSoup.find_all("p")[5]
    college = collegeSection.find('strong',text = re.compile('College'))
    playerCollege = ''
    if college is not None:
        playerCollege = collegeSection.find_all('a', href=True)[0].get_text()

    print("{} {} {} {} {} {} {} {} {} {} {} {}".format(playerId,missingPlayerLink,first.strip(),last.strip(),team,birthDate,jerseyNumber,False,position,playerCollege,jerseyNumber, birthDate,height,weight ))

    if not team:
        cursor.execute("INSERT INTO Players ([ProReferenceKey],[ProReferenceURL], [FullName], [FirstName], [LastName], [JerseyNumber], [Rookie], [Position], [College], [DateOfBirth], [Height], [Weight], [TradingBlock], [Retired], [WeeksLeftOnIR]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", playerId,nameLink['href'],fullName,first.strip(),last.strip(),jerseyNumber,False,position,playerCollege,birthDate,height,weight,False, retired,0 )
        cursor.commit()
    else:
        cursor.execute("INSERT INTO Players ([ProReferenceKey],[ProReferenceURL], [FullName],[FirstName], [LastName], [NFLTeamId], [JerseyNumber], [Rookie], [Position], [College], [DateOfBirth], [Height], [Weight], [TradingBlock], [Retired],[WeeksLeftOnIR]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", playerId,nameLink['href'],fullName,first.strip(),last.strip(),team,jerseyNumber,False,position,playerCollege,birthDate,height,weight,False,retired,0 )
        cursor.commit()
    
cursor.close()

https://www.pro-football-reference.com/players/A/AbbrJa00.htm
AbbrJa00 /players/A/AbbrJa00.htm Jared Abbrederis   10 False WR  10 
https://www.pro-football-reference.com/players/A/AbduAm00.htm
AbduAm00 /players/A/AbduAm00.htm Ameer Abdullah   20 False RB Nebraska 20 
https://www.pro-football-reference.com/players/A/AdamDa01.htm
AdamDa01 /players/A/AdamDa01.htm Davante Adams   17 False WR Fresno St. 17 
https://www.pro-football-reference.com/players/A/AdamJe01.htm
AdamJe01 /players/A/AdamJe01.htm Jerell Adams   89 False TE  89 
https://www.pro-football-reference.com/players/A/AdamJo03.htm
AdamJo03 /players/A/AdamJo03.htm Josh Adams   36 False RB  36 
https://www.pro-football-reference.com/players/A/AdamRo01.htm
AdamRo01 /players/A/AdamRo01.htm Rodney Adams   13 False WR  13 
https://www.pro-football-reference.com/players/A/AddiBr01.htm
AddiBr01 /players/A/AddiBr01.htm Bralon Addison   45 False RB  45 
https://www.pro-football-reference.com/players/A/AdebQu00.htm
AdebQu00 /players/A/Adeb

KeyboardInterrupt: 

In [44]:
cursor = connection.cursor()

playerUrl = 'https://www.pro-football-reference.com/years/2022/draft.htm'
page = requests.get(playerUrl)
soup = BeautifulSoup(page.text, 'html.parser')
rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')
for data in rows:
    cols = data.find_all('td')
    if len(cols) > 0:
        position = cols[3].get_text()
        if position in positionList:
            teamLink = cols[1].find("a", href=True)
            nameLink = cols[2].find("a", href=True)        
            fullName = nameLink.get_text()
            playerId = nameLink['href'].split('/')[3].split('.')[0]
            
            names = nameLink.get_text().split(" ")
            first = names[0]
            last = ''
            for name in names[1:]:
                last = last + ' ' + name
            print("{} {}".format(first.strip(), last.strip()))
            
            playerUrl = 'https://www.pro-football-reference.com{}'.format(nameLink['href'])
            playerPage = requests.get(playerUrl)
            playerSoup = BeautifulSoup(playerPage.text, 'html.parser')
            jerseyContainer = playerSoup.find("div", {"class": "uni_holder"}).find_all('text');
            jerseyNumber = ''
            if len(jerseyContainer) > 0:
                jerseyNumber = jerseyContainer[-1].get_text()
            
            team = ''
            t = teamLink.get_text()
            team = GetTeamId(t.upper())
#             currentTeam = playerSoup.find("span", {"itemprop": "affiliation"})
#             if currentTeam is not None:
#                 t = currentTeam.find("a", href=True)['href'].split('/')[2]
#                 team = GetTeamId(t.upper())
            birthDate = ''
            dob = playerSoup.find("span", {"itemprop": "birthDate"})
            if dob is not None:
                birthDate = dob['data-birth']
            height = ''
            h = playerSoup.find("span", {"itemprop": "height"})
            if h is not None:
                height = h.get_text()
            weight = ''
            w = playerSoup.find("span", {"itemprop": "weight"})
            if w is not None:
                weight = w.get_text()
                
            collegeLink = cols[26].find("a", href=True)
            playerCollege = collegeLink.get_text()
#             collegeSection = playerSoup.find_all("p")[5]
#             college = collegeSection.find('strong',text = re.compile('College'))
#             playerCollege = ''
#             if college is not None:
#                 playerCollege = collegeSection.find_all('a', href=True)[0].get_text()
                
            print("{} {} {} {} {} {} {} {} {} {} {} {}".format(playerId,nameLink['href'],first.strip(),last.strip(),team,birthDate,jerseyNumber,True,position,playerCollege,jerseyNumber, birthDate,height,weight ))
            
            if not team:
                cursor.execute("INSERT INTO Players ([ProReferenceKey],[ProReferenceURL], [FullName],[FirstName], [LastName], [JerseyNumber], [Rookie], [Position], [College], [DateOfBirth], [Height], [Weight],[TradingBlock], [Retired],[WeeksLeftOnIR]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", playerId,nameLink['href'],fullName,first.strip(),last.strip(),jerseyNumber,True,position,playerCollege,birthDate,height,weight,False,False,0 )
                cursor.commit()
            else:
                cursor.execute("INSERT INTO Players ([ProReferenceKey],[ProReferenceURL], [FullName],[FirstName], [LastName], [NFLTeamId], [JerseyNumber], [Rookie], [Position], [College], [DateOfBirth], [Height], [Weight],[TradingBlock], [Retired],[WeeksLeftOnIR]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", playerId,nameLink['href'],fullName,first.strip(),last.strip(),team,jerseyNumber,True,position,playerCollege,birthDate,height,weight,False,False,0 )
                cursor.commit()
                
cursor.close()

Drake London
LondDr00 /players/L/LondDr00.htm Drake London 2   True WR USC  
Garrett Wilson
WilsGa00 /players/W/WilsGa00.htm Garrett Wilson 25   True WR Ohio St.  
Chris Olave
OlavCh00 /players/O/OlavCh00.htm Chris Olave 23   True WR Ohio St.  
Jameson Williams
WillJa11 /players/W/WillJa11.htm Jameson Williams 11   True WR Alabama  
Jahan Dotson
DotsJa00 /players/D/DotsJa00.htm Jahan Dotson 32   True WR Penn St.  
Treylon Burks
BurkTr00 /players/B/BurkTr00.htm Treylon Burks 31   True WR Arkansas  
Kenny Pickett
PickKe00 /players/P/PickKe00.htm Kenny Pickett 27   True QB Pittsburgh  
Christian Watson
WatsCh00 /players/W/WatsCh00.htm Christian Watson 12   True WR North Dakota St.  
Breece Hall
HallBr03 /players/H/HallBr03.htm Breece Hall 25   True RB Iowa St.  
Kenneth Walker
WalkKe00 /players/W/WalkKe00.htm Kenneth Walker 29   True RB Michigan St.  
Wan'Dale Robinson
RobiWa01 /players/R/RobiWa01.htm Wan'Dale Robinson 24   True WR Kentucky  
John Metchie
MetcJo00 /players/M/MetcJo00.htm 

In [45]:
cursor = connection.cursor()
years = [2022]
loopSeasonId = 5
i = 785
for currentYear in years:
    url = 'https://www.pro-football-reference.com/years/{}/games.htm'.format(currentYear)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

    if currentYear == 2022:
        for data in rows:
            week = data.find_all('th')[0].get_text()
            if week == 'Week':
                continue
            cols = data.find_all('td')
            link = cols[1].find("a", href=True)
            if link is None:
                continue
            dateLink = link['href']
            date = dateLink.split('/')[2].split('.')[0][:-4]
            year = date[:-4]
            month = date[4:-2]
            day = date[6:]
            awayLink = cols[2].find("a", href=True)
            away = awayLink['href'].split('/')[2].upper()
            awayId = GetTeamId(away)

            homeLink = cols[5].find("a", href=True)
            home = homeLink['href'].split('/')[2].upper()
            homeId = GetTeamId(home)
            time = cols[7].get_text()
            cursor.execute("INSERT INTO NFLSchedule ([Id],[SeasonId], [Week], [HomeTeamId], [AwayTeamId], [StartTime],[BoxscoreLink]) VALUES (?,?,?,?,?,?,?)",i,loopSeasonId,week,homeId,awayId,"{}-{}-{} {}".format(year, month, day, time),dateLink)
            cursor.commit()
            print("Week {} : {} ({}) @ {} ({}) {}/{}/{} {} {}".format(week, away, awayId, home,homeId, month,day,year, time, dateLink))
            i += 1
    if currentYear < 2021:
        for data in rows:
            week = data.find_all('th')[0].get_text()
            if week == 'Week':
                continue
            cols = data.find_all('td')
            link = cols[6].find("a", href=True)
            if link is None:
                continue
            
            dateLink = link['href']

            date = dateLink.split('/')[2].split('.')[0][:-4]

            awayWinner = cols[4].get_text() == '@'


            year = date[:-4]
            month = date[4:-2]
            day = date[6:]
            winnerLink = cols[3].find("a", href=True)
            winner = winnerLink['href'].split('/')[2].upper()
            winnerId = GetTeamId(winner)

            loserLink = cols[5].find("a", href=True)
            loser = loserLink['href'].split('/')[2].upper()
            loserId = GetTeamId(loser)

            if awayWinner == True:
                away = winner
                awayId = winnerId
                home = loser
                homeId = loserId
            if awayWinner == False:
                home = winner
                homeId = winnerId
                away = loser
                awayId = loserId

            time = cols[7].get_text()
            isInt = IsAnInt(week)
            if isInt:
                cursor.execute("INSERT INTO NFLSchedule ([Id],[SeasonId], [Week], [HomeTeamId], [AwayTeamId], [StartTime],[BoxscoreLink]) VALUES (?,?,?,?,?,?,?)",i,loopSeasonId,week,homeId,awayId,"{}-{}-{}".format(year, month, day),dateLink)
                cursor.commit()
                print("Week {} : {} ({}) @ {} ({}) {}/{}/{}  {}".format(week, away, awayId, home,homeId, month,day,year, dateLink))
                print(awayWinner)
                i += 1
    loopSeasonId += 1
cursor.close()


Week 1 : BUF (4) @ RAM (19) 09/08/2022 8:20 PM /boxscores/202209080ram.htm
Week 1 : NOR (23) @ ATL (2) 09/11/2022 1:00 PM /boxscores/202209110atl.htm
Week 1 : CLE (8) @ CAR (5) 09/11/2022 1:00 PM /boxscores/202209110car.htm
Week 1 : SFO (28) @ CHI (6) 09/11/2022 1:00 PM /boxscores/202209110chi.htm
Week 1 : PIT (27) @ CIN (7) 09/11/2022 1:00 PM /boxscores/202209110cin.htm
Week 1 : PHI (26) @ DET (11) 09/11/2022 1:00 PM /boxscores/202209110det.htm
Week 1 : CLT (14) @ HTX (13) 09/11/2022 1:00 PM /boxscores/202209110htx.htm
Week 1 : NWE (22) @ MIA (20) 09/11/2022 1:00 PM /boxscores/202209110mia.htm
Week 1 : RAV (3) @ NYJ (25) 09/11/2022 1:00 PM /boxscores/202209110nyj.htm
Week 1 : JAX (15) @ WAS (32) 09/11/2022 1:00 PM /boxscores/202209110was.htm
Week 1 : KAN (16) @ CRD (1) 09/11/2022 4:25 PM /boxscores/202209110crd.htm
Week 1 : GNB (12) @ MIN (21) 09/11/2022 4:25 PM /boxscores/202209110min.htm
Week 1 : NYG (24) @ OTI (31) 09/11/2022 4:25 PM /boxscores/202209110oti.htm
Week 1 : RAI (17) @ 

In [26]:
def BuildSeasonStats(year, seasonId):
    url = 'https://www.pro-football-reference.com/years/{}/passing.htm'.format(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

    passingCursor = connection.cursor()

    for data in rows:
        cols = data.find_all('td')
        if len(cols) > 0:  
            nameLink = cols[0].find_all('a', href=True)[0];
            name = nameLink.get_text()

            playerId = nameLink['href'].split('/')[3].split('.')[0]

            passingCursor.execute("SELECT Id FROM Players WHERE ProReferenceKey = ?", playerId)
            for row in passingCursor.fetchall():
                playerId = row[0]

            gamesPlayed = cols[4].get_text()
            gamesStarted = cols[5].get_text()

            comp = cols[7].get_text()
            attempts = cols[8].get_text()
            yards = cols[10].get_text()
            td = cols[11].get_text()
            inter = cols[13].get_text()
            qbr = cols[22].get_text()

            if isinstance(playerId, int):
                statId = -1
                passingCursor.execute("SELECT * FROM SeasonPlayerStats WHERE SeasonId = ? AND PlayerId = ?",seasonId, playerId)
                for row in passingCursor.fetchall():
                    statId = row[0]
                if statId > 0:
                    #update
                    passingCursor.execute("UPDATE [dbo].[SeasonPlayerStats] ([SeasonId], [PlayerId], [GamesPlayed], [GamesStarted], [PassYards], [Completions],[PassTDs],[Interceptions],[PassAttempts],[QBRating]) VALUES (?,?,?,?,?,?,?,?,?,?)", seasonId, playerId,gamesPlayed,gamesStarted,yards,comp,td,inter,attempts,qbr)
                    passingCursor.commit()
                else:
                    #add
                    passingCursor.execute("INSERT INTO SeasonPlayerStats ([SeasonId], [PlayerId], [GamesPlayed], [GamesStarted], [PassYards], [Completions],[PassTDs],[Interceptions],[PassAttempts],[QBRating]) VALUES (?,?,?,?,?,?,?,?,?,?)", seasonId, playerId,gamesPlayed,gamesStarted,yards,comp,td,inter,attempts,qbr)
                    passingCursor.commit()
            print("Name: {} ({}) - Games: ({}/{})  Comp: {}  Attempt: {}  Yards: {} TDs: {} INTs: {} QBR: {}".format(name.strip(), playerId,gamesStarted,gamesPlayed,comp,attempts,yards,td,inter,qbr))

    passingCursor.close()
    
    url = 'https://www.pro-football-reference.com/years/{}/rushing.htm'.format(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

    rushingCursor = connection.cursor()

    for data in rows:
        cols = data.find_all('td')
        if len(cols) > 0:  
            nameLink = cols[0].find_all('a', href=True)[0];
            name = nameLink.get_text()

            playerId = nameLink['href'].split('/')[3].split('.')[0]

            rushingCursor.execute("SELECT Id FROM Players WHERE ProReferenceKey = ?", playerId)
            for row in rushingCursor.fetchall():
                playerId = row[0]

            gamesPlayed = cols[4].get_text()
            gamesStarted = cols[5].get_text()

            attempts = cols[6].get_text()
            yards = cols[7].get_text()
            td = cols[8].get_text()
            average = cols[12].get_text()
            fumbles = cols[13].get_text()

            if isinstance(playerId, int):
                statId = -1
                rushingCursor.execute("SELECT * FROM SeasonPlayerStats WHERE SeasonId = ? AND PlayerId = ?",seasonId, playerId)
                for row in rushingCursor.fetchall():
                    statId = row[0]
                if statId > 0:
    #                 update
                    rushingCursor.execute("UPDATE SeasonPlayerStats SET [GamesStarted] = ?,[GamesPlayed] = ?, [RushAttempts] = ?,[RushYards] = ?,[RushTDs] = ?,[Fumbles] = ?,[RushAverage] = ? WHERE SeasonId = ? AND PlayerId = ?",gamesStarted,gamesPlayed,attempts,yards,td,fumbles,average,seasonId, playerId)
                    rushingCursor.commit()
                else:
    #                 add
                    rushingCursor.execute("INSERT INTO SeasonPlayerStats ([SeasonId], [PlayerId],[GamesStarted],[GamesPlayed], [RushAttempts],[RushYards],[RushTDs],[Fumbles],[RushAverage]) VALUES (?,?,?,?,?,?,?,?,?)", seasonId, playerId,gamesStarted,gamesPlayed,attempts,yards,td,fumbles,average)
                    rushingCursor.commit()
            print("Name: {} ({}) - Games: ({}/{})  Rush Attempts: {} Yards: {} TDs: {} Fumbles: {} Average: {}".format(name.strip(), playerId,gamesStarted,gamesPlayed,attempts,yards,td,fumbles,average))

    rushingCursor.close()
    
    url = 'https://www.pro-football-reference.com/years/{}/receiving.htm'.format(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

    recCursor = connection.cursor()

    for data in rows:
        cols = data.find_all('td')
        if len(cols) > 0:  
            nameLink = cols[0].find_all('a', href=True)[0];
            name = nameLink.get_text()

            playerId = nameLink['href'].split('/')[3].split('.')[0]

            recCursor.execute("SELECT Id FROM Players WHERE ProReferenceKey = ?", playerId)
            for row in recCursor.fetchall():
                playerId = row[0]

            gamesPlayed = cols[4].get_text()
            gamesStarted = cols[5].get_text()

            targets = cols[6].get_text()
            rec = cols[7].get_text()
            yards = cols[9].get_text()
            td = cols[11].get_text()
            average = cols[16].get_text()
            fumbles = cols[17].get_text()

            if isinstance(playerId, int):
                statId = -1
                recCursor.execute("SELECT * FROM SeasonPlayerStats WHERE SeasonId = ? AND PlayerId = ?",seasonId, playerId)
                for row in recCursor.fetchall():
                    statId = row[0]
                if statId > 0:
    #                 update
                    recCursor.execute("UPDATE SeasonPlayerStats SET [GamesStarted] = ?,[GamesPlayed] = ?, [Targets] = ?, [Receptions] = ?, [RecYards] = ?,[RecTDs] = ?,[Fumbles] = ?,[RecAverage] = ? WHERE SeasonId = ? AND PlayerId = ?",gamesStarted,gamesPlayed,targets,rec,yards,td,fumbles,average,seasonId, playerId)
                    recCursor.commit()
                else:
    #                 add
                    recCursor.execute("INSERT INTO SeasonPlayerStats ([SeasonId], [PlayerId],[GamesStarted],[GamesPlayed], [Targets],[Receptions],[RecYards],[RecTDs],[Fumbles],[RecAverage]) VALUES (?,?,?,?,?,?,?,?,?,?)", seasonId, playerId,gamesStarted,gamesPlayed,targets,rec,yards,td,fumbles,average)
                    recCursor.commit()
            print("Name: {} ({}) - Games: ({}/{})  Targets: {} Catches: {} Yards: {} TDs: {} Fumbles: {} Average: {}".format(name.strip(), playerId,gamesStarted,gamesPlayed,targets,rec,yards,td,fumbles,average))

    recCursor.close()
    

    url = 'https://www.pro-football-reference.com/years/{}/returns.htm'.format(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

    returnCursor = connection.cursor()

    for data in rows:
        cols = data.find_all('td')
        if len(cols) > 0:  
            nameLink = cols[0].find_all('a', href=True)[0];
            name = nameLink.get_text()

            playerId = nameLink['href'].split('/')[3].split('.')[0]

            returnCursor.execute("SELECT Id FROM Players WHERE ProReferenceKey = ?", playerId)
            for row in returnCursor.fetchall():
                playerId = row[0]

            gamesPlayed = cols[4].get_text()
            gamesStarted = cols[5].get_text()

            prAttempts = cols[6].get_text() 
            prYards = cols[7].get_text()
            prTDs = cols[8].get_text()

            krAttempts = cols[11].get_text() 
            krYards = cols[12].get_text()
            krTDs = cols[13].get_text()

            if isinstance(playerId, int):
                statId = -1
                returnCursor.execute("SELECT * FROM SeasonPlayerStats WHERE SeasonId = ? AND PlayerId = ?",seasonId, playerId)
                for row in returnCursor.fetchall():
                    statId = row[0]
                if statId > 0:
    #                 update
                    returnCursor.execute("UPDATE SeasonPlayerStats SET [GamesStarted] = ?,[GamesPlayed] = ?, [PRAttempted] = ?,[PRYards] = ?,[PRTDs] = ?,[KRAttempted] = ?,[KRYards] = ?, [KRTDs] = ? WHERE SeasonId = ? AND PlayerId = ?",gamesStarted,gamesPlayed,prAttempts,prYards,prTDs,krAttempts,krYards,krTDs, seasonId, playerId)
                    returnCursor.commit()
                else:
    #                 add
                    returnCursor.execute("INSERT INTO SeasonPlayerStats ([SeasonId], [PlayerId],[GamesStarted],[GamesPlayed], [PRAttempted],[PRYards],[PRTDs],[KRAttempted],[KRYards], [KRTDs]) VALUES (?,?,?,?,?,?,?,?,?,?)", seasonId, playerId,gamesStarted,gamesPlayed,prAttempts,prYards,prTDs,krAttempts,krYards,krTDs)
                    returnCursor.commit()
            print("Name: {} ({}) - Games: ({}/{})  Punts: {} Attempts, {} Yards, {} TDs  Kick: {} Attempts, {} Yards, {} TDs".format(name.strip(), playerId,gamesStarted,gamesPlayed,prAttempts,prYards,prTDs,krAttempts,krYards,krTDs))

    returnCursor.close()
    
    url = 'https://www.pro-football-reference.com/years/{}/scoring.htm'.format(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

    scoringCursor = connection.cursor()

    for data in rows:
        cols = data.find_all('td')
        if len(cols) > 0:  
            nameLink = cols[0].find_all('a', href=True)[0];
            name = nameLink.get_text()

            playerId = nameLink['href'].split('/')[3].split('.')[0]

            scoringCursor.execute("SELECT Id FROM Players WHERE ProReferenceKey = ?", playerId)
            for row in scoringCursor.fetchall():
                playerId = row[0]

            gamesPlayed = cols[4].get_text()
            gamesStarted = cols[5].get_text()
            made = cols[14].get_text()

            if isinstance(playerId, int):
                statId = -1
                scoringCursor.execute("SELECT * FROM SeasonPlayerStats WHERE SeasonId = ? AND PlayerId = ?",seasonId, playerId)
                for row in scoringCursor.fetchall():
                    statId = row[0]
                if statId > 0:
    #                 update
                    scoringCursor.execute("UPDATE SeasonPlayerStats SET [GamesStarted] = ?,[GamesPlayed] = ?, [TwoPointConv] = ? WHERE SeasonId = ? AND PlayerId = ?",gamesStarted,gamesPlayed,made, seasonId, playerId)
                    scoringCursor.commit()
                else:
    #                 add
                    scoringCursor.execute("INSERT INTO SeasonPlayerStats ([SeasonId], [PlayerId],[GamesStarted],[GamesPlayed], [TwoPointConv]) VALUES (?,?,?,?,?)", seasonId, playerId,gamesStarted,gamesPlayed,made)
                    scoringCursor.commit()
            print("Name: {} ({}) Games ({}/{}) - Made:{}".format(name.strip(),playerId,gamesStarted,gamesPlayed,made))

    scoringCursor.close()
    
    url = 'https://www.pro-football-reference.com/years/{}/kicking.htm'.format(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

    kickingCursor = connection.cursor()

    for data in rows:
        cols = data.find_all('td')
        if len(cols) > 0:  
            nameLink = cols[0].find_all('a', href=True)[0];
            name = nameLink.get_text()

            playerId = nameLink['href'].split('/')[3].split('.')[0]

            kickingCursor.execute("SELECT Id FROM Players WHERE ProReferenceKey = ?", playerId)
            for row in kickingCursor.fetchall():
                playerId = row[0]

            gamesPlayed = cols[4].get_text()
            gamesStarted = cols[5].get_text()

            fgAttempt = cols[16].get_text()
            fgMade = cols[17].get_text()

            xpAttempt = cols[20].get_text()
            xpMade = cols[21].get_text()

            fg50Att = cols[14].get_text()
            fg50Made = cols[15].get_text()

            fg40Att = cols[12].get_text()
            fg40Made = cols[13].get_text()

            fg30Att = cols[10].get_text()
            fg30Made = cols[11].get_text()        

            fg20Att = cols[8].get_text()
            fg20Made = cols[9].get_text()        

            fg19Att = cols[6].get_text()
            fg19Made = cols[7].get_text()

            if isinstance(playerId, int):
                statId = -1
                kickingCursor.execute("SELECT * FROM SeasonPlayerStats WHERE SeasonId = ? AND PlayerId = ?",seasonId, playerId)
                for row in kickingCursor.fetchall():
                    statId = row[0]
                if statId > 0:
    #                 update
                    kickingCursor.execute("UPDATE SeasonPlayerStats SET [GamesStarted] = ?,[GamesPlayed] = ?, [FieldGoalsMade] = ?, [FieldGoalsAttempted] = ?, [FieldGoals1to19Attempted] = ?,[FieldGoals1to19Made] = ?,[FieldGoals20to29Attempted] = ?,[FieldGoals20to29Made] = ?,[FieldGoals30to39Attempted] = ?,[FieldGoals30to39Made] = ?,[FieldGoals40to49Attempted] = ?,[FieldGoals40to49Made] = ?,[FieldGoals50PlusAttempted] = ?,[FieldGoals50PlusMade] = ?,[PATAttempted] = ?,[PATMade] = ?  WHERE SeasonId = ? AND PlayerId = ?",gamesStarted,gamesPlayed,fgMade,fgAttempt,fg19Att,fg19Made,fg20Att,fg20Made,fg30Att,fg30Made,fg40Att,fg40Made,fg50Att,fg50Made,xpAttempt,xpMade,seasonId, playerId)
                    kickingCursor.commit()
                else:
    #                 add
                    kickingCursor.execute("INSERT INTO SeasonPlayerStats ([SeasonId], [PlayerId],[GamesStarted],[GamesPlayed],[FieldGoalsMade],[FieldGoalsAttempted],[FieldGoals1to19Attempted],[FieldGoals1to19Made],[FieldGoals20to29Attempted],[FieldGoals20to29Made],[FieldGoals30to39Attempted],[FieldGoals30to39Made],[FieldGoals40to49Attempted],[FieldGoals40to49Made],[FieldGoals50PlusAttempted],[FieldGoals50PlusMade],[PATAttempted] ,[PATMade]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", seasonId, playerId,gamesStarted,gamesPlayed,fgMade,fgAttempt,fg19Att,fg19Made,fg20Att,fg20Made,fg30Att,fg30Made,fg40Att,fg40Made,fg50Att,fg50Made,xpAttempt,xpMade)
                    kickingCursor.commit()
            print("Name: {} ({}) Games ({}/{}) - FG:{}/{}  XP:{}/{} 1-19:{}/{} 20-29:{}/{} 30-39:{}/{} 40-49:{}/{} 50+:{}/{}".format(name.strip(),playerId,gamesStarted,gamesPlayed,fgMade,fgAttempt,xpAttempt,xpMade,fg19Made,fg19Att,fg20Made,fg20Att,fg30Made,fg30Att,fg40Made,fg40Att,fg50Made,fg50Att))

    kickingCursor.close()


In [28]:
BuildSeasonStats(2021,4)

ProgrammingError: ('42000', "[42000] [Microsoft][SQL Server Native Client 11.0][SQL Server]Incorrect syntax near '('. (102) (SQLExecDirectW); [42000] [Microsoft][SQL Server Native Client 11.0][SQL Server]Statement(s) could not be prepared. (8180)")

In [46]:
def GetTeamId(team):
    return {
        'ARI':1,
        'Arizona Cardinals':1,
        'CRD':1,
        'ATL':2,
        'Atlanta Falcons':2,
        'BAL':3,
        'RAV':3,
        'Baltimore Ravens':3,
        'BUF':4,
        'Buffalo Bills':4,
        'CAR':5,
        'Carolina Panthers':5,
        'CHI':6,
        'Chicago Bears':6,
        'CIN':7,
        'Cincinnati Bengals':7,
        'CLE':8,
        'Cleveland Browns':8,
        'DAL':9,
        'Dallas Cowboys':9,
        'DEN':10,
        'Denver Broncos':10,
        'DET':11,
        'Detroit Lions':11,
        'GB':12,
        'Green Bay Packers':12,
        'GNB':12,
        'HOU':13,
        'HTX':13,
        'Houston Texans':13,
        'IND':14,
        'Indianapolis Colts':14,
        'CLT':14,
        'JAC':15,
        'JAX':15,
        'Jacksonville Jaguars':15,
        'KAN':16,
        'KC':16,
        'Kansas City Chiefs':16,
        'LV':17,
        'LVR':17,
        'RAI':17,
        'Las Vegas Raiders':17,
        'LAC':18,
        'Los Angeles Chargers':18,
        'SDG':18,
        'LAR':19,
        'Los Angeles Rams':19,
        'RAM':19,
        'MIA':20,
        'Miami Dolphins':20,
        'MIN':21,
        'Minnesota Vikings':21,
        'NWE':22,
        'New England Patriots':22,
        'NE':22,
        'NOR':23,
        'New Orleans Saints':23,
        'NO':23,
        'NYG':24,
        'New York Giants':24,
        'NYJ':25,
        'New York Jets':25,
        'PHI':26,
        'Philadelphia Eagles':26,
        'PIT':27,
        'Pittsburgh Steelers':27,
        'SF':28,
        'San Francisco 49ers':28,
        'SFO':28,
        'SEA':29,
        'Seattle Seahawks':29,
        'TB':30,
        'Tampa Bay Buccaneers':30,
        'TAM':30,
        'TEN':31,
        'Tennessee Titans':31,
        'OTI':31,
        'WAS':32,
        'Washington Football Team':32,
        'Washington Commanders':32,
        '':'',
        '2TM':''
    }[team]

In [47]:
url = 'https://www.fantasypros.com/nfl/projections/qb.php'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

passingCursor = connection.cursor()

for data in rows:
    cols = data.find_all('td')
    nameLink = cols[0].find("a", href=True)
    fullName = nameLink.get_text()
    names = nameLink.get_text().split(" ")
    first = names[0]
    last = ''
    for name in names[1:]:
        if (name == 'II'):
            continue
        last = last + ' ' + name
        
    last = last.strip()

    nflTeam = cols[0].getText().split(' ')[-2]
    passAttempt = float(cols[1].getText().replace(',', ''))
    complete = float(cols[2].getText().replace(',', ''))
    passYards = float(cols[3].getText().replace(',', ''))
    passTD = float(cols[4].getText().replace(',', ''))
    passInt = float(cols[5].getText().replace(',', ''))
    rushAtt = float(cols[6].getText().replace(',', ''))
    rushYards = float(cols[7].getText().replace(',', ''))
    rushTD = float(cols[8].getText().replace(',', ''))
    fumLost = float(cols[9].getText().replace(',', ''))
    
    sql = "SELECT * FROM Players WHERE FirstName  LIKE ? AND LastName LIKE ? AND NFLTeamId = ? AND Position = 'QB' "
    firstNameParam = f'%{first}%'    
    lastNameParam = f'%{last}%'
    nflTeamId = GetTeamId(nflTeam)
    dbPlayer = passingCursor.execute(sql, firstNameParam, lastNameParam, nflTeamId)
    for row in dbPlayer.fetchall():
        playerId = row[0]
    print(fullName)
    
    updateDB = True
    sqlCheck = "SELECT * FROM SeasonProjectedStats WHERE PlayerId = ? AND SeasonId = 5"
    statCheck = passingCursor.execute(sqlCheck, playerId)
    for row in statCheck.fetchall():
        print(row[0])
        updateDB = False
        
    if updateDB:
        passingCursor.execute("INSERT INTO SeasonProjectedStats ([SeasonId], [PlayerId],[PassYards], [Completions],[PassTDs],[Interceptions],[PassAttempts],[RushYards],[RushTDs],[Fumbles],[KickingPoints],[DefensivePoints]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)", 5, playerId,passYards,complete,passTD,passInt,passAttempt,rushYards,rushTD,fumLost,0,0)
        passingCursor.commit()

    print(GetTeamId(nflTeam))
    
passingCursor.close()

Josh Allen
4
Patrick Mahomes II
16
Justin Herbert
18
Lamar Jackson
3
Kyler Murray
1
Jalen Hurts
26
Tom Brady
30
Dak Prescott
9
Joe Burrow
7
Aaron Rodgers
12
Matthew Stafford
19
Trey Lance
28
Russell Wilson
10
Kirk Cousins
21
Derek Carr
17
Tua Tagovailoa
20
Ryan Tannehill
31
Trevor Lawrence
15
Justin Fields
6
Daniel Jones
24
Matt Ryan
14
Mac Jones
22
Jameis Winston
23
Carson Wentz
32
Jared Goff
11
Zach Wilson
25
Baker Mayfield
5
Davis Mills
13
Marcus Mariota
2
Mitch Trubisky
27
Drew Lock
29
Jacoby Brissett
8
Deshaun Watson
8
Kenny Pickett
27
Geno Smith
29
Desmond Ridder
2
Andy Dalton
23
Joe Flacco
25
Sam Darnold
5
Mason Rudolph
27
Tyrod Taylor
24
Taysom Hill
23
Tyler Huntley
3
Jimmy Garoppolo
28
Teddy Bridgewater
20
Joshua Dobbs
8
Trevor Siemian
6
Gardner Minshew II
26
Taylor Heinicke
32
Cooper Rush
9
Malik Willis
31
Colt McCoy
1
Josh Johnson
10
Jordan Love
12
Kyle Allen
13
Blaine Gabbert
30
Brian Hoyer
22
C.J. Beathard
15
Kyle Trask
30
Case Keenum
4
Jarrett Stidham
17
Sean Mannion
21
J

In [48]:
url = 'https://www.fantasypros.com/nfl/projections/rb.php'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

cursor = connection.cursor()

for data in rows:
    cols = data.find_all('td')
    nameLink = cols[0].find("a", href=True)
    fullName = nameLink.get_text()
    names = nameLink.get_text().split(" ")
    first = names[0]
    last = ''
    for name in names[1:]:
        if (name == 'II'):
            continue
        last = last + ' ' + name
        
    last = last.strip()

    nflTeam = cols[0].getText().split(' ')[-2]
    rushAttempt = float(cols[1].getText().replace(',', ''))
    rushYards = float(cols[2].getText().replace(',', ''))
    rushTD = float(cols[3].getText().replace(',', ''))
    rec = float(cols[4].getText().replace(',', ''))
    recYards = float(cols[5].getText().replace(',', ''))
    recTD = float(cols[6].getText().replace(',', ''))
    fumbles = float(cols[7].getText().replace(',', ''))

    sql = "SELECT * FROM Players WHERE FirstName  LIKE ? AND LastName LIKE ? AND NFLTeamId = ? AND Position = 'RB' "
    firstNameParam = f'%{first}%'    
    lastNameParam = f'%{last}%'
    nflTeamId = GetTeamId(nflTeam)
    dbPlayer = cursor.execute(sql, firstNameParam, lastNameParam, nflTeamId)
    for row in dbPlayer.fetchall():
        playerId = row[0]
    print(fullName)
    print("{} {} {} {}".format(rushAttempt, rushYards, rec, fumbles))
    updateDB = True
    sqlCheck = "SELECT * FROM SeasonProjectedStats WHERE PlayerId = ? AND SeasonId = 5"
    statCheck = cursor.execute(sqlCheck, playerId)
    for row in statCheck.fetchall():
        updateDB = False
        
    if updateDB:
        cursor.execute("INSERT INTO SeasonProjectedStats ([SeasonId], [PlayerId],[RushAttempts], [RushYards],[RushTDs],[Receptions],[RecYards],[RecTDs],[Fumbles],[KickingPoints],[DefensivePoints]) VALUES (?,?,?,?,?,?,?,?,?,?,?)", 5, playerId,rushAttempt,rushYards,rushTD,rec,recYards,recTD,fumbles,0,0)
        cursor.commit()
    print(GetTeamId(nflTeam))
    
cursor.close()

Jonathan Taylor
326.1 1602.9 42.4 2.3
14
Derrick Henry
366.2 1640.4 27.4 2.0
31
Austin Ekeler
221.0 983.6 74.4 1.5
18
Christian McCaffrey
238.6 1061.6 89.0 1.4
5
Joe Mixon
286.9 1202.6 47.1 1.6
7
Dalvin Cook
276.2 1241.8 44.9 2.5
21
Alvin Kamara
219.2 963.2 60.0 1.2
23
Najee Harris
298.5 1184.0 60.7 1.6
27
Leonard Fournette
230.9 961.8 61.1 1.3
30
James Conner
226.5 927.4 44.1 1.6
1
Aaron Jones
193.6 920.0 60.9 1.5
12
Nick Chubb
247.7 1230.4 25.2 1.6
8
Javonte Williams
232.5 1025.5 45.9 1.7
10
Saquon Barkley
244.9 1027.2 57.6 1.5
24
D'Andre Swift
216.2 914.4 65.4 1.9
11
David Montgomery
234.8 948.3 44.4 1.4
6
Ezekiel Elliott
232.3 971.0 40.8 2.1
9
Breece Hall
219.6 991.7 39.3 1.8
25
Cam Akers
236.3 1027.5 34.9 3.3
19
J.K. Dobbins
195.9 948.9 33.2 1.0
3
Elijah Mitchell
239.1 1027.9 25.8 1.2
28
Damien Harris
215.1 977.2 18.2 1.4
22
Josh Jacobs
223.2 941.3 38.0 1.9
17
Miles Sanders
204.4 1043.3 31.2 1.6
26
Travis Etienne Jr.
177.2 807.4 48.8 1.7
15
Clyde Edwards-Helaire
180.8 797.9 36.3 1

In [49]:
url = 'https://www.fantasypros.com/nfl/projections/wr.php'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

cursor = connection.cursor()

for data in rows:
    cols = data.find_all('td')
    nameLink = cols[0].find("a", href=True)
    fullName = nameLink.get_text()
    names = nameLink.get_text().split(" ")
    first = names[0]
    last = ''
    for name in names[1:]:
        if (name == 'II'):
            continue
        last = last + ' ' + name
        
    last = last.strip()

    nflTeam = cols[0].getText().split(' ')[-2]
    rec = float(cols[1].getText().replace(',', ''))
    recYards = float(cols[2].getText().replace(',', ''))
    recTD = float(cols[3].getText().replace(',', ''))
    rushAttempt = float(cols[4].getText().replace(',', ''))
    rushYards = float(cols[5].getText().replace(',', ''))
    rushTD = float(cols[6].getText().replace(',', ''))

    fumbles = float(cols[7].getText().replace(',', ''))

    sql = "SELECT * FROM Players WHERE FirstName  LIKE ? AND LastName LIKE ? AND NFLTeamId = ? AND Position = 'WR' "
    firstNameParam = f'%{first}%'    
    lastNameParam = f'%{last}%'
    nflTeamId = GetTeamId(nflTeam)
    dbPlayer = cursor.execute(sql, firstNameParam, lastNameParam, nflTeamId)
    for row in dbPlayer.fetchall():
        playerId = row[0]
    print(fullName)
    print("{} {} {} {}".format(rec, recYards, recTD, fumbles))
    updateDB = True
    sqlCheck = "SELECT * FROM SeasonProjectedStats WHERE PlayerId = ? AND SeasonId = 5"
    statCheck = cursor.execute(sqlCheck, playerId)
    for row in statCheck.fetchall():
        updateDB = False
        
    if updateDB:
        cursor.execute("INSERT INTO SeasonProjectedStats ([SeasonId], [PlayerId],[RushAttempts], [RushYards],[RushTDs],[Receptions],[RecYards],[RecTDs],[Fumbles],[KickingPoints],[DefensivePoints]) VALUES (?,?,?,?,?,?,?,?,?,?,?)", 5, playerId,rushAttempt,rushYards,rushTD,rec,recYards,recTD,fumbles,0,0)
        cursor.commit()
    print(GetTeamId(nflTeam))
    
cursor.close()

Cooper Kupp
123.0 1599.7 12.4 0.6
19
Ja'Marr Chase
91.7 1417.5 10.5 0.6
7
Justin Jefferson
103.2 1506.8 9.6 0.6
21
Deebo Samuel
70.7 1052.8 6.1 0.7
28
Davante Adams
106.4 1322.2 9.2 0.5
17
Stefon Diggs
102.1 1256.9 8.9 0.5
4
Mike Evans
82.7 1138.9 10.1 0.5
30
CeeDee Lamb
91.1 1233.9 7.6 0.5
9
Tyreek Hill
92.1 1149.4 8.0 0.5
20
Tee Higgins
85.3 1195.2 7.8 0.5
7
A.J. Brown
77.6 1134.6 8.1 0.5
26
Mike Williams
74.6 1098.1 7.9 0.4
18
Michael Pittman Jr.
91.1 1126.7 6.6 0.6
14
DJ Moore
87.0 1146.7 5.6 0.6
5
Brandin Cooks
92.7 1119.8 6.2 0.5
13
Keenan Allen
99.6 1086.5 6.9 0.5
18
Gabriel Davis
61.5 960.9 8.8 0.1
4
DK Metcalf
78.5 1057.2 6.8 0.6
29
Diontae Johnson
94.8 1045.6 6.1 1.0
27
Terry McLaurin
78.5 1071.2 5.9 0.4
32
Courtland Sutton
70.8 1004.2 6.8 0.5
10
Jaylen Waddle
89.5 999.1 6.1 0.6
20
Michael Thomas
85.5 995.1 6.5 0.5
23
Amon-Ra St. Brown
87.6 951.3 6.0 0.5
11
Jerry Jeudy
75.3 1024.6 5.8 0.6
10
Darnell Mooney
77.9 1029.5 4.8 0.5
6
Tyler Lockett
74.2 995.7 5.6 0.5
29
Chris Godwin

In [50]:
url = 'https://www.fantasypros.com/nfl/projections/te.php'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

cursor = connection.cursor()

for data in rows:
    cols = data.find_all('td')
    nameLink = cols[0].find("a", href=True)
    fullName = nameLink.get_text()
    names = nameLink.get_text().split(" ")
    first = names[0]
    last = ''
    for name in names[1:]:
        if (name == 'II'):
            continue
        last = last + ' ' + name
        
    last = last.strip()

    nflTeam = cols[0].getText().split(' ')[-2]
    rec = float(cols[1].getText().replace(',', ''))
    recYards = float(cols[2].getText().replace(',', ''))
    recTD = float(cols[3].getText().replace(',', ''))

    fumbles = float(cols[4].getText().replace(',', ''))

    sql = "SELECT * FROM Players WHERE FirstName  LIKE ? AND LastName LIKE ? AND NFLTeamId = ? AND Position = 'TE' "
    firstNameParam = f'%{first}%'    
    lastNameParam = f'%{last}%'
    nflTeamId = GetTeamId(nflTeam)
    dbPlayer = cursor.execute(sql, firstNameParam, lastNameParam, nflTeamId)
    for row in dbPlayer.fetchall():
        playerId = row[0]
    print(fullName)
    print("{} {} {} {}".format(rec, recYards, recTD, fumbles))
    updateDB = True
    sqlCheck = "SELECT * FROM SeasonProjectedStats WHERE PlayerId = ? AND SeasonId = 5"
    statCheck = cursor.execute(sqlCheck, playerId)
    for row in statCheck.fetchall():
        updateDB = False
        
    if updateDB:
        cursor.execute("INSERT INTO SeasonProjectedStats ([SeasonId], [PlayerId],[Receptions],[RecYards],[RecTDs],[Fumbles],[KickingPoints],[DefensivePoints]) VALUES (?,?,?,?,?,?,?,?)", 5, playerId,rec,recYards,recTD,fumbles,0,0)
        cursor.commit()

    print(GetTeamId(nflTeam))
    
cursor.close()

Travis Kelce
99.4 1179.3 9.3 0.5
16
Mark Andrews
96.1 1195.0 8.9 0.5
3
Kyle Pitts
73.1 997.8 4.6 0.5
2
Darren Waller
77.5 931.0 5.1 0.5
17
George Kittle
71.2 870.8 5.4 0.5
28
Dalton Schultz
77.4 805.2 6.2 0.4
9
Dallas Goedert
63.5 794.5 5.0 0.5
26
T.J. Hockenson
67.6 693.3 5.1 0.4
11
Hunter Henry
53.8 618.0 6.0 0.1
22
Dawson Knox
49.1 593.6 6.2 0.1
4
Zach Ertz
64.7 662.9 4.8 0.4
1
Mike Gesicki
61.5 671.1 4.0 0.4
20
Noah Fant
60.7 653.6 3.8 0.4
29
Tyler Higbee
59.2 595.5 4.7 0.4
19
David Njoku
53.2 602.9 4.3 0.4
8
Pat Freiermuth
62.7 579.6 4.8 0.5
27
Cole Kmet
60.8 632.8 3.7 0.5
6
Irv Smith Jr.
50.4 556.7 4.5 0.1
21
Gerald Everett
55.0 567.0 4.5 0.9
18
Robert Tonyan
45.8 520.5 4.5 0.1
12
Albert Okwuegbunam
49.4 509.4 4.7 0.8
10
Logan Thomas
48.6 508.0 4.5 0.4
32
Evan Engram
52.7 541.6 3.5 0.4
15
Austin Hooper
50.9 497.5 4.1 0.4
31
Hayden Hurst
42.0 441.2 4.0 0.5
7
Mo Alie-Cox
34.9 422.1 3.2 0.1
14
Brevin Jordan
41.3 418.0 2.9 0.1
13
Adam Trautman
36.1 374.4 2.8 0.4
23
Kyle Rudolph
33.2 

In [51]:
url = 'https://www.fantasypros.com/nfl/projections/k.php'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

cursor = connection.cursor()

for data in rows:
    cols = data.find_all('td')
    nameLink = cols[0].find("a", href=True)
    fullName = nameLink.get_text()
    names = nameLink.get_text().split(" ")
    first = names[0]
    last = ''
    for name in names[1:]:
        if (name == 'II'):
            continue
        last = last + ' ' + name
        
    last = last.strip()

    nflTeam = cols[0].getText().split(' ')[-2]
    fg = float(cols[1].getText().replace(',', ''))
    fga = float(cols[2].getText().replace(',', ''))
    xp = float(cols[3].getText().replace(',', ''))

    points = float(cols[4].getText().replace(',', ''))

    sql = "SELECT * FROM Players WHERE FirstName  LIKE ? AND LastName LIKE ? AND NFLTeamId = ? AND Position = 'K' "
    firstNameParam = f'%{first}%'    
    lastNameParam = f'%{last}%'
    nflTeamId = GetTeamId(nflTeam)
    dbPlayer = cursor.execute(sql, firstNameParam, lastNameParam, nflTeamId)
    for row in dbPlayer.fetchall():
        playerId = row[0]
    print(fullName)
    print("{} {} {} {}".format(rec, recYards, recTD, fumbles))
    updateDB = True
    sqlCheck = "SELECT * FROM SeasonProjectedStats WHERE PlayerId = ? AND SeasonId = 5"
    statCheck = cursor.execute(sqlCheck, playerId)
    for row in statCheck.fetchall():
        updateDB = False
        
    if updateDB:
        cursor.execute("INSERT INTO SeasonProjectedStats ([SeasonId], [PlayerId],[FieldGoalsMade],[FieldGoalsAttempted],[PATMade],[KickingPoints],[DefensivePoints]) VALUES (?,?,?,?,?,?,?)", 5, playerId,fg,fga,xp,points,0)
        cursor.commit()

    print(GetTeamId(nflTeam))
    
cursor.close()

Justin Tucker
0.0 0.0 0.0 0.0
3
Harrison Butker
0.0 0.0 0.0 0.0
16
Daniel Carlson
0.0 0.0 0.0 0.0
17
Matt Gay
0.0 0.0 0.0 0.0
19
Ryan Succop
0.0 0.0 0.0 0.0
30
Tyler Bass
0.0 0.0 0.0 0.0
4
Greg Joseph
0.0 0.0 0.0 0.0
21
Matt Prater
0.0 0.0 0.0 0.0
1
Mason Crosby
0.0 0.0 0.0 0.0
12
Evan McPherson
0.0 0.0 0.0 0.0
7
Nick Folk
0.0 0.0 0.0 0.0
22
Rodrigo Blankenship
0.0 0.0 0.0 0.0
14
Robbie Gould
0.0 0.0 0.0 0.0
28
Brandon McManus
0.0 0.0 0.0 0.0
10
Dustin Hopkins
0.0 0.0 0.0 0.0
18
Chris Boswell
0.0 0.0 0.0 0.0
27
Randy Bullock
0.0 0.0 0.0 0.0
31
Younghoe Koo
0.0 0.0 0.0 0.0
2
Jake Elliott
0.0 0.0 0.0 0.0
26
Wil Lutz
0.0 0.0 0.0 0.0
23
Jason Sanders
0.0 0.0 0.0 0.0
20
Graham Gano
0.0 0.0 0.0 0.0
24
Cairo Santos
0.0 0.0 0.0 0.0
6
Austin Seibert
0.0 0.0 0.0 0.0
11
Joey Slye
0.0 0.0 0.0 0.0
32
Zane Gonzalez
0.0 0.0 0.0 0.0
5
Cade York
0.0 0.0 0.0 0.0
8
Jason Myers
0.0 0.0 0.0 0.0
29
Ka'imi Fairbairn
0.0 0.0 0.0 0.0
13
Greg Zuerlein
0.0 0.0 0.0 0.0
25
Riley Patterson
0.0 0.0 0.0 0.0
11
Brett 

In [52]:
url = 'https://www.fantasypros.com/nfl/projections/dst.php'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
rows = soup.find_all("table")[0].find_all('tbody')[0].find_all('tr')

cursor = connection.cursor()

for data in rows:
    cols = data.find_all('td')
    nameLink = cols[0].find("a", href=True)
    fullName = nameLink.get_text()

    sacks = float(cols[1].getText().replace(',', ''))
    dInt = float(cols[2].getText().replace(',', ''))
    fr = float(cols[3].getText().replace(',', ''))
    td = float(cols[5].getText().replace(',', ''))
    safety = float(cols[6].getText().replace(',', ''))
    pa = float(cols[7].getText().replace(',', ''))
    yards = float(cols[8].getText().replace(',', ''))
    points = float(cols[9].getText().replace(',', ''))

    nflTeamId = GetTeamId(fullName) * -1

    playerId = nflTeamId


    updateDB = True
    sqlCheck = "SELECT * FROM SeasonProjectedStats WHERE PlayerId = ? AND SeasonId = 5"
    statCheck = cursor.execute(sqlCheck, playerId)
    for row in statCheck.fetchall():
        updateDB = False
        
    if updateDB:
        print("{} {} {} {}".format(fullName, nflTeamId, sacks, td))
        cursor.execute("INSERT INTO SeasonProjectedStats ([SeasonId], [PlayerId],[Sacks],[DefensiveInt],[DefensiveFumbleRecovery],[DefensiveTD],[Safety],[PointsAllowed],[YardsAllowed],[KickingPoints],[DefensivePoints]) VALUES (?,?,?,?,?,?,?,?,?,?,?)", 5, nflTeamId,sacks,dInt,fr,td,safety,pa,yards,0,points)
        cursor.commit()


    
cursor.close()

Tampa Bay Buccaneers -30 44.2 2.9
Buffalo Bills -4 43.3 2.8
Pittsburgh Steelers -27 48.9 2.5
Dallas Cowboys -9 40.0 3.0
Green Bay Packers -12 43.0 2.8
Los Angeles Rams -19 43.4 2.8
New Orleans Saints -23 43.9 2.6
Miami Dolphins -20 44.2 2.6
Los Angeles Chargers -18 42.1 2.7
San Francisco 49ers -28 44.7 2.6
Indianapolis Colts -14 35.9 2.7
New England Patriots -22 40.5 2.5
Minnesota Vikings -21 42.0 2.8
Cleveland Browns -8 42.8 2.5
Kansas City Chiefs -16 37.8 2.8
Arizona Cardinals -1 38.9 2.4
Denver Broncos -10 40.6 2.6
Tennessee Titans -31 41.8 2.5
Cincinnati Bengals -7 39.8 2.4
Washington Commanders -32 39.7 2.5
Baltimore Ravens -3 38.8 2.6
Chicago Bears -6 41.9 2.3
Carolina Panthers -5 39.4 2.4
Philadelphia Eagles -26 39.6 2.7
New York Giants -24 38.2 2.3
Seattle Seahawks -29 36.9 2.2
Houston Texans -13 35.4 2.3
Jacksonville Jaguars -15 38.9 2.2
Las Vegas Raiders -17 36.3 2.2
New York Jets -25 36.2 2.3
Detroit Lions -11 35.4 2.2
Atlanta Falcons -2 31.0 2.1
